In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Retrieve Data

In [2]:
from stockMarket.core import get_tickers_from_index

tickers = get_tickers_from_index('sp500')

In [3]:
from stockMarket.api import populate_contracts

contracts = populate_contracts(tickers)
# contracts = populate_contracts(tickers, update_period="now")
import numpy as np
import pandas as pd

from stockMarket.core import Screener, LimitScreenerObject, EqualityScreenerObject
from stockMarket.core import growth_total

screener = Screener(contracts)

bank_screener = EqualityScreenerObject('fin_statement_type', equal_to="BANK", description="Bank")
not_bank_screener = EqualityScreenerObject('fin_statement_type', not_equal_to="BANK", description="Bank")

dividend_yield_screener = LimitScreenerObject('dividend_yield', min_value=0.1, description="Dividend Yield")
payout_screener = LimitScreenerObject('payout_ratio', max_value=150, description="Payout Ratio")
price_screener = LimitScreenerObject('price', max_value=40, description="Price")
trailing_pe_screener = LimitScreenerObject('trailing_pe', min_value=0, max_value=30, description="Trailing PE")
forward_pe_screener = LimitScreenerObject('forward_pe', min_value=0, max_value=30, description="Forward PE")
revenue_screener = LimitScreenerObject('revenue', min_value=0, lambda_func= lambda x: growth_total(x)[0], description="Revenue Growth")
operating_cashflow_screener = LimitScreenerObject('operating_cashflow', min_value=0, description="Operating Cashflow")
equity_ratio_screener_not_bank = LimitScreenerObject('equity_ratio', min_value=30, description="Equity Ratio", screener_object=not_bank_screener)
equity_ratio_screener_bank = LimitScreenerObject('equity_ratio', min_value=8, description="Equity Ratio", screener_object=bank_screener)
ebitda_margin_screener = LimitScreenerObject('ebitda_margin', min_value=15, description="EBITDA Margin")
net_income_screener = LimitScreenerObject('net_income', min_value=0, lambda_func=np.min, description="Min Net Income")

screener_list = [
    dividend_yield_screener,
    payout_screener,
    price_screener,
    trailing_pe_screener,
    forward_pe_screener,
    revenue_screener,
    operating_cashflow_screener,
    equity_ratio_screener_not_bank,
    equity_ratio_screener_bank,
    ebitda_margin_screener,
    net_income_screener
]

screened_contracts = screener.screen(screener_list)

/home/jag/Trading/stockMarket/stockMarket/yfinance/_common.py:36: UserWarning: Not all tickers are in calendar and the selected update period indicates that the calendar is up to date. Therefore the calendar will not be updated and the missing tickers will be remain empty. To enforce an update the calendar set update to 'now'
  warnings.warn(
/home/jag/Trading/stockMarket/stockMarket/yfinance/_common.py:36: UserWarning: Not all tickers are in calendar and the selected update period indicates that the calendar is up to date. Therefore the calendar will not be updated and the missing tickers will be remain empty. To enforce an update the calendar set update to 'now'
  warnings.warn(


### Perform Screening

In [9]:
import numpy as np
import pandas as pd

from stockMarket.core import Ranking, RankingObject, RangeRankingObject
   

equity_ratio_ranking = RangeRankingObject(
    lambda x: x.equity_ratio[0], 
    "Equity Ratio",
    [(30, None, 1), (None, 10, -1)]
)
netto_margin_ranking = RangeRankingObject(
    lambda x: x.netto_margin[0],
    "Netto Margin",
    [(10, 20, 1), (20, None, 2)]
)
return_on_assets_ranking = RangeRankingObject(
    lambda x: x.return_on_assets[0],
    "Return on Assets",
    [(5, 10, 1), (10, None, 2)]
)
goodwill_ranking = RangeRankingObject(
    lambda x: x.goodwill_ratio[0],
    "Goodwill",
    [(30, None, -5)]
)
ranking_list = [
    equity_ratio_ranking,
    netto_margin_ranking,
    return_on_assets_ranking,
    goodwill_ranking,
]

ranking = Ranking(screened_contracts, ranking_list)
ranking.rank()
ranking.ranking.to_csv("ranking_contracts.csv", sep='\t', encoding='utf-8', float_format='%.2f')
ranking.ranking

,Name,Sector,Total Score,Equity Ratio,Equity Ratio Score,Netto Margin,Netto Margin Score,Return on Assets,Return on Assets Score,Goodwill,Goodwill Score
ALB,Albemarle Corporation,Basic Materials,5,51.645687,1,36.745598,2,17.402466,2,20.264344,0
GOOG,Alphabet Inc.,Communication Services,5,70.125717,1,21.203807,2,16.418809,2,11.306140,0
GOOGL,Alphabet Inc.,Communication Services,5,70.125717,1,21.203807,2,16.418809,2,11.306140,0
META,"Meta Platforms, Inc.",Communication Services,5,66.704119,1,28.982521,2,17.027040,2,13.484540,0
CTRA,Coterra Energy Inc.,Energy,5,62.865932,1,44.912164,2,20.169693,2,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...
WAB,Westinghouse Air Brake Technologies Corporation,Industrials,-4,54.558220,1,7.569959,0,3.418665,0,84.220946,-5
EQIX,"Equinix, Inc.",Real Estate,-4,37.960028,1,9.697574,0,2.323747,0,49.141611,-5
TYL,"Tyler Technologies, Inc.",Technology,-4,55.987957,1,8.876859,0,3.503849,0,94.852859,-5
STT,State Street Corporation,Financial Services,-5,8.006176,-1,10.584776,1,0.653977,0,31.980335,-5


### Write Email

In [10]:
from stockMarket.utils import write_email

emails = ["benjaminlantschner@gmail.com", "97gamjak@gmail.com"]
subject = "Stock Market Ranking"
body = ""
attachment = "ranking_contracts.csv"
write_email(emails, subject, body, attachment)